In [1]:
import os
import langchain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.callbacks import get_openai_callback
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain.agents import AgentExecutor, create_react_agent, create_openai_tools_agent, Tool, tool
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_community.utilities.openweathermap import OpenWeatherMapAPIWrapper
from langchain.document_loaders import WebBaseLoader
from dotenv import load_dotenv

load_dotenv(dotenv_path=f"{os.path.abspath('')}/../../backend/.env.dev", verbose=True)

True

In [2]:
# tools

weather_search_util = OpenWeatherMapAPIWrapper(openweathermap_api_key=os.environ["OPENWEATHERMAP_API_KEY"])
google_search_util = GoogleSearchAPIWrapper(google_api_key=os.environ["GOOGLE_API_KEY"], google_cse_id=os.environ["GOOGLE_CSE_ID"], k=5)

@tool
def website_scrape_util(url: str) -> str:
    """Scrape the provided web page for detailed information."""
    try:
        loader = WebBaseLoader(url)
        docs = loader.load()
        return "\n\n".join(
            [f'<Document name="{doc.metadata.get("title", "")}">\n{doc.page_content}\n</Document>' for doc in docs]
        )
    except Exception as e:
        return "Error happened when using tool: " + str(e) + ". You should check your input, it must be a valid URL."


def get_search_google_tool():
    return Tool(
        name="search_google",
        description="Search Google using a query and get a list of results. Recommended to use the language related to the context of the conversation to get the best results.",
        func=google_search_util.run)


def get_search_weather_tool():
    return Tool(
        name="search_weather",
        description="Get current weather information of specific location. The input format is 'City, Country'.",
        func=weather_search_util.run)


def get_scrape_website_tool():
    return Tool(
        name="scrape_website",
        description="Scrape specific web page using an URL you provided for detailed information. You should use the search_google tool first, and then take the webpage URL you like to provide to this tool.",
        func=website_scrape_util,
    )



In [3]:
def create_agent(llm: ChatOpenAI, system_prompt: str, tools: list):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [4]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

research_agent = create_agent(llm, "You are responsible for searching and validating data related to Vietnamese tourism, ensuring that the information provided is accurate and up-to-date. Whether the Human needs specific details about a destination, cultural insights, historical data, or statistical information, you are here to assist.",
                              [get_search_google_tool(), get_search_weather_tool(), get_scrape_website_tool()],)

planning_agent = create_agent(llm, "You are responsible for tasks related to planning tours, visits, food places, and other activities for tourists based on their preferences, duration of stay, location, and number of people. Assistant needs to create well-organized and thoughtful plans, taking into account the Human's preferences and any specific requirements. You can ask the Human for more info, or confirmations about your schedule.", [get_search_google_tool(), get_search_weather_tool(), get_scrape_website_tool()]
)

reply_agent = create_agent(llm, "You need to make a summary about every step the team is working on, who is the currently talking, and what they are talking about. You need to make every line in a format: <AGENT - CONTEXT - THEIR MESSAGES>", [get_search_google_tool()])

In [5]:
members = ["Researcher", "Planner"]

system_prompt = (
    "You are a supervisor tasked with managing a conversation with the Human, and between the"
    " following workers: {members}. Given the following conversation history and the current user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished or you need to ask the Human,"
    " make the reply and respond with FINISH."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            },
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatOpenAI(model="gpt-4o")

supervisor_chain = (
        prompt
        | llm.bind_functions(functions=[function_def], function_call="route")
        | JsonOutputFunctionsParser()
)

In [6]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [AIMessage(content=result["output"], name=name)]}

In [7]:
import functools
import operator
from typing import Sequence, TypedDict, Annotated

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import END, StateGraph, START


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str


research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")
planning_node = functools.partial(agent_node, agent=planning_agent, name="Planner")
reply_node = functools.partial(agent_node, agent=reply_agent, name="ReplyAgent")


workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("Planner", planning_node)
workflow.add_node("ReplyAgent", reply_node)
workflow.add_node("supervisor", supervisor_chain)

In [8]:
for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = "ReplyAgent"
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
workflow.add_edge("ReplyAgent", END)
# Finally, add entrypoint
workflow.add_edge(START, "supervisor")

graph = workflow.compile()

In [12]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Bạn biết thời tiết hiện tại ở Đà nẵng là như nào không, và có những quán ăn nào ngon ở đấy không nhỉ ?")
        ]
    }, debug=True
):
    if "__end__" not in s:
        print(s)
        print(type(s))
        print(s.keys())
        print("----")

[0:tasks] Starting step 0 with 1 task:
- __start__ -> {'messages': [HumanMessage(content='Bạn biết thời tiết hiện tại ở Đà nẵng là như nào không, và có những quán ăn nào ngon ở đấy không nhỉ ?')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [HumanMessage(content='Bạn biết thời tiết hiện tại ở Đà nẵng là như nào không, và có những quán ăn nào ngon ở đấy không nhỉ ?')]
[1:tasks] Starting step 1 with 1 task:
- supervisor -> {'messages': [HumanMessage(content='Bạn biết thời tiết hiện tại ở Đà nẵng là như nào không, và có những quán ăn nào ngon ở đấy không nhỉ ?')],
 'next': None}
{'supervisor': {'next': 'Researcher'}}
<class 'langgraph.pregel.io.AddableUpdatesDict'>
dict_keys(['supervisor'])
----
[1:writes] Finished step 1 with writes to 1 channel:
- next -> 'Researcher'
[2:tasks] Starting step 2 with 1 task:
- Researcher -> {'messages': [HumanMessage(content='Bạn biết thời tiết hiện tại ở Đà nẵng là như nào không, và có những quán ăn nào ngon ở đấy không nhỉ ?')],
 